In [2]:
import torch
import torch.nn as nn

from GINN.problem_sampler import ProblemSampler
from train.train_utils.latent_sampler import sample_new_z
from utils import get_stateless_net_with_partials, get_model
from neural_clbf.controllers.simple_neural_cbf_controller import SimpleNeuralCBFController
from neural_clbf.systems.simple3d import Simple3DRobot
from configs.get_adapter_config import build_config
from configs.get_config import get_config_from_yml
from models.model_utils import tensor_product_xz
from train.train_utils.loss_optims import GradNormBalancer
from torch.utils.tensorboard import SummaryWriter

import subprocess
import time
from datetime import datetime
import os
from copy import deepcopy
from tqdm import trange
from collections import defaultdict
import yaml

from adapter_model import LossTimer, ConditionalSIRENWithAdapter, AdapterMLP, create_adapter_mlp


/sfs/weka/scratch/rhm4nj/cral/cral-ginn/ginn/neural_clbf/neural_clbf/systems/f16.py:22: UserWarning: Could not import F16 module; is AeroBench installed?
  warn("Could not import F16 module; is AeroBench installed?")
/home/rhm4nj/.local/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/sfs/weka/scratch/rhm4nj/cral/cral-ginn/ginn/neural_clbf/neural_clbf/experiments/__init__.py:44: UserWarning: Could not import HW module; is ROS installed?
  warn("Could not import HW module; is ROS installed?")


In [3]:
def load_yaml_to_dict(filename: str) -> dict:
    with open(filename, 'r') as file:
        data = yaml.safe_load(file)
    return data

In [6]:
adapter_config_path = "/scratch/rhm4nj/cral/cral-ginn/ginn/configs/adapter_config.yml"
device='cuda'

# Load and initialize models
def get_adapter_model(config_path):
    config = load_yaml_to_dict(adapter_config_path)
    config_siren = get_config_from_yml(config["paths"]["siren_config_path"])
    config_siren["device"] = device

    siren_model = get_model(config_siren).to(device)
    siren_model.load_state_dict(torch.load(config["paths"]["pretrained_siren_path"], map_location=device))

    final_layer_size = list(siren_model.network.children())[-3].out_features
    layer_sizes = [final_layer_size] + config["training"]["adapter_mid_layers"]
    activation_name = config["training"]["activation_name"]

    adapter_model = create_adapter_mlp(layer_sizes, activation_name=activation_name, siren_config=config_siren).to(device)
    model = ConditionalSIRENWithAdapter(siren_model, adapter_model).to(device)
    return model

model = get_adapter_model(adapter_config_path)

Loaded config from /scratch/rhm4nj/cral/cral-ginn/ginn/configs/adapter_config.yml
